# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Alternatywy

Zamiast bazowych sieci rekurencyjnych lub sieci LSTM (i ich [modyfikacji](https://en.wikipedia.org/wiki/Long_short-term_memory)) można zastosować różne alternatywy, np. sieci <b>Gated Recurrent Unit<b> (GRU):
    
[![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Gated_Recurrent_Unit.svg)](https://en.wikipedia.org/wiki/Gated_recurrent_unit)

Lub inne modele skonstruowane do rozwiązywania specyficznych problemów, np. [uczenia na szeregach czasowych.](https://github.com/sdobber/FluxArchitectures)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy, params
using StatsBase: wsample
using Base.Iterators: partition
using BSON
using JLD2
using CUDA

In [2]:
use_cuda = true

true

In [3]:
 if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

[ Info: Training on GPU


Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [4]:
isfile("w_poszukiwaniu.txt") ||
        download("https://raw.githubusercontent.com/bartoszpankratz/221660-0553-Aproksymacja/master/6.%20Sieci%20Rekurencyjne/w_poszukiwaniu.txt","w_poszukiwaniu.txt")

true

In [5]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [6]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [7]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [8]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen)) |> device;
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen)) |> device;

In [33]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 512),
    LSTM(512, 256),
    Dense(256, 128, relu),
    Dense(128, 64, relu),
    Dense(64, N),
    softmax) |> device

function loss(xs, ys, ϵ = 1.0f-32)
  l = sum(crossentropy.(broadcast(x -> m(x) .+ ϵ, xs), ys))
  Flux.reset!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
    model = cpu(m)
    Flux.reset!(model)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, model(onehot(c, alphabet)))
      end
    return String(take!(buf))
end

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#31 (generic function with 1 method)

In [34]:
loss(Xs[5], Ys[5])

481.27957f0

In [35]:
sample(m, alphabet, 50)

"ëNDZZêPçUn1ęP5Él*èC-uTph*_'vM3äLrїêmô.çyRiÉa4:üfkà"

In [36]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:25
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        char_model = cpu.(params(m))
        BSON.@save "char_model.bson" char_model
        jldsave("char_model.jld2"; char_model)
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

[ Info: Beginning training loop...
[ Info: Epoch: 1


loss(Xs[5], Ys[5]) = 480.3568f0
IM„áŻ/Kq7N?WL18ütEèżĆnôjŹ1ńż—ëÀjN)ńù85OBŻŚüÉkNą”Lr2iÉÀö
 ŚGD°áùąŻzbP-À—. ”­écsâbąu1g­ëü,x?dzŃQ Kobi)
loss(Xs[5], Ys[5]) = 325.27155f0
U!twaj t weckltze ceoęi  kaod cćzsłykręiah i,aueck jpad,ńryłeanoć wrta o y,oisdnS  nilwk aymdtaatise
loss(Xs[5], Ys[5]) = 325.03308f0
M jćjilezesay amy  rk isewgonaspwnżasrgti ky a  ij ormtwApssłao oko ewzkonu  s m onśri,uy oZ y o a s


[ Info: New best result: 324.86514
[ Info: Epoch: 2


loss(Xs[5], Ys[5]) = 324.91858f0
f wa ey zr,edaełąnaścżroż jr,lsw,t.c i amwz pwead
łyn, joo jod,yadg zemowowcndosl eó,omcnr jjięyaays
loss(Xs[5], Ys[5]) = 324.823f0
d p—ąąup  oysngka sd eśaprrajaóukjoi:zy ooksj,nę peuiwer.aei trenzeaęmżny iłpntt ywo  a irtkazr paza
loss(Xs[5], Ys[5]) = 324.94537f0
qmęempzeaoja pee ać iendn grs Pknithąm ay er owib r  nmm Rnsą wpry  cęwc śaw  d.hyybww,ztragnnosi iz


[ Info: Epoch: 3


...

[ Info: New best result: 184.26135
[ Info: Epoch: 24


loss(Xs[5], Ys[5]) = 184.27945f0
siężej myiała ją odwając i onagrony serta lepa „mu kolać częłamarponiej, lzad wątrzych uł Czukazgewi
loss(Xs[5], Ys[5]) = 181.54135f0
Czem zamarowalie w to.

Zą, aby nie jej lidy zawać w za bała z w ją matarz przeciw zrazy, onlck, ch 
loss(Xs[5], Ys[5]) = 183.62143f0
Fap. Osić obrycie sobie światów że jedni, ródem, a nie, że się mięskną. Albertyna wóka, czas sobie w
loss(Xs[5], Ys[5]) = 184.00995f0
Horek, ego komusisiry zaze przerłając w o co innych poą cją się na matku rozojał sąwie, jakże, że ma


[ Info: Epoch: 25


loss(Xs[5], Ys[5]) = 185.20535f0
Óloza — i um gas się, ceub domkoboczucia! Modstanałą zację swoi i kowąc oskące i zd mroczem Chadzy c
loss(Xs[5], Ys[5]) = 183.05292f0
za, bo onek pi sposobniona masm kogasty — przyjaciółu, że wiedziałach ile dlatego domóni docy od tyc
loss(Xs[5], Ys[5]) = 181.36703f0
êowe żuś pamie byłyć rozko mnieji i Tutent, zbytej do hającą mnie Alercie, nieokocjomiło spociążych 


[ Info: New best result: 181.71532


In [40]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 512),
    LSTM(512, 256),
    Dense(256, 128, relu),
    Dense(128, 64, relu),
    Dense(64, N),
    softmax) |> device

ps = JLD2.load("char_model.jld2", "char_model")

Flux.loadparams!(m, device.(ps))

In [41]:
@show loss(Xs[5], Ys[5])

loss(Xs[5], Ys[5]) = 181.71532f0


181.71532f0

In [43]:
sample(m, alphabet, 50)

"gdy sranęło czasoz pan jej ją wiel wyda. jak drzec"

## Dodatkowa praca domowa

Zmodyfikuj kod tak, aby poprawić jakość generowanego tekstu <b>(5 punktów)</b>.